In [1]:
#main
import pandas as pd
import numpy as np
import math
import sklearn
import re

#graphic
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
movie_overview_2022 = pd.read_csv('./movies_2022.csv', index_col='Unnamed: 0')
movie_detail_2022 = pd.read_csv('./movie_details_2022.csv', index_col='Unnamed: 0')
movie_award_2022 = pd.read_csv('./awards_2022.csv', index_col='Unnamed: 0')
top_1000_movies = pd.read_csv('./imdb_top_1000.csv')


In [4]:
movie_overview_2022 = movie_overview_2022.reset_index(drop=True)

In [5]:
movie_detail_2022.drop(['title','movie_id','movie_imdb_link' ], axis=1, inplace=True) # drop duplicate columns

In [6]:
movie_award_2022.drop(['title','movie_id'], axis=1, inplace=True) # drop duplicate columns

In [7]:
movies_df = pd.concat([movie_overview_2022, movie_award_2022, movie_detail_2022 ],axis = 1)


In [8]:
movies_df.shape

(477, 26)

In [9]:
def separete_awards(df):
    df['total_award_nominations'] = '' #creating empty columns
    df['total_award_wins'] = ''

    for i in df.index:
        if df['awards_total'][i] != '0':
            pattern= '\d+'
            numbers = re.findall(pattern,df['awards_total'][i])
            df['total_award_wins'][i] = numbers[0]      
            df['total_award_nominations'][i] = numbers[1]         
      
        else:
            df['total_award_wins'][i] = df['awards_total'][i]
            df['total_award_nominations'][i] = df['awards_total'][i]
    
    return df

In [10]:
movies_df_v1 = separete_awards(movies_df)

/var/folders/33/p_3l01b14g96rn22vzwly2g00000gn/T/ipykernel_64253/2812189874.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_award_wins'][i] = numbers[0]
/var/folders/33/p_3l01b14g96rn22vzwly2g00000gn/T/ipykernel_64253/2812189874.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_award_nominations'][i] = numbers[1]
/var/folders/33/p_3l01b14g96rn22vzwly2g00000gn/T/ipykernel_64253/2812189874.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

In [11]:
def separete_genre(df):
    df['primary_genre'] = '' #creating empty columns
    df['secondary_genre'] = ''
    
    pattern = "[\w']+"
    
    for i in df.index:
        if df['genre'][i] != '[]':
            g = re.findall(pattern,df['genre'][i])
            
            if len(g)>1:
                df['primary_genre'][i] = g[0]      
                df['secondary_genre'][i] = g[1]
            else:
                df['primary_genre'][i] = df['genre'][i]
                df['secondary_genre'][i] = 'Other'
        else:
            df['primary_genre'][i] = ''
            df['secondary_genre'][i] = ''
    
    return df
    

In [12]:
movies_df_v2 = separete_genre(movies_df_v1)

/var/folders/33/p_3l01b14g96rn22vzwly2g00000gn/T/ipykernel_64253/4184901931.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['primary_genre'][i] = g[0]
/var/folders/33/p_3l01b14g96rn22vzwly2g00000gn/T/ipykernel_64253/4184901931.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['secondary_genre'][i] = g[1]
/var/folders/33/p_3l01b14g96rn22vzwly2g00000gn/T/ipykernel_64253/4184901931.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [13]:
movies_df_v2['certificate']= movies_df_v2['certificate'].replace({
    'R':'Adult',
    'PG-13':'+13/14', 
    'TV-13':'+13/14',
    'TV-MA':'Adult',
    'PG': 'Parental Guidance',
    'TV-14': '+13/14',
    'Unrated': 'Not Rated',
    'Approved': 'Not Rated',
    'TV-PG': 'Parental Guidance',
    'TV-G': 'Kids',
    'G': 'Suitable for all',
    'TV-Y7': 'Kids',
    '18': 'Adult',
    'TV-Y': 'Kids',
    'TV-Y7-FV': 'Kids',
    'T':'Adult',
    'M': 'Adult'
})

In [14]:
def drop_k(df):
    
    pattern = "K"
    
    for i in df.index:
        if df['num_user_reviews'][i] != '[]':
            review = re.findall(pattern,df['num_user_reviews'][i]) 
            if len(review)>0:
                a = df['num_user_reviews'][i].replace('K','')    
                df['num_user_reviews'][i] = float(a)*1000
                
            else:
                df['num_user_reviews'][i] = df['num_user_reviews'][i]
        else:
            df['num_user_reviews'][i] = 0
    
    df['num_user_reviews'] =  pd.to_numeric(df['num_user_reviews'], errors='coerce')
    
    return df
    

In [15]:
movies_df_v2 = drop_k(movies_df_v2)

/var/folders/33/p_3l01b14g96rn22vzwly2g00000gn/T/ipykernel_64253/3710906870.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['num_user_reviews'][i] = float(a)*1000
/var/folders/33/p_3l01b14g96rn22vzwly2g00000gn/T/ipykernel_64253/3710906870.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['num_user_reviews'][i] = df['num_user_reviews'][i]


In [16]:
movies_df_v2['num_critic_reviews'] =  pd.to_numeric(movies_df_v2['num_critic_reviews'], errors='coerce')

In [17]:
movies_df_v2['release_date'].value_counts(dropna=False)

October 7, 2022       13
March 18, 2022        12
August 26, 2022       11
September 30, 2022    10
August 12, 2022       10
September 16, 2022    10
June 3, 2022          10
June 17, 2022          9
July 15, 2022          9
August 5, 2022         9
February 11, 2022      9
January 14, 2022       9
July 29, 2022          8
September 23, 2022     8
April 8, 2022          8
April 1, 2022          8
April 29, 2022         8
October 14, 2022       7
October 28, 2022       7
September 2, 2022      7
January 28, 2022       6
May 6, 2022            6
October 21, 2022       5
April 15, 2022         5
June 24, 2022          5
November 4, 2022       5
May 20, 2022           5
September 9, 2022      5
November 18, 2022      4
February 25, 2022      4
May 27, 2022           4
February 17, 2022      4
2022                   4
March 4, 2022          4
October 4, 2022        4
July 8, 2022           4
February 18, 2022      4
July 1, 2022           4
February 4, 2022       4
March 25, 2022         4


In [18]:
movies_df_v2['release_date']= movies_df_v2['release_date'].replace({
    '2022':'',
    '2023':''
})

In [19]:
movies_df_v2['release_date']=pd.to_datetime(movies_df_v2['release_date'], errors='coerce')

In [20]:
movies_df_v2['release_date'].value_counts(dropna=False)

2022-10-07    13
2022-03-18    12
2022-08-26    11
2022-09-16    10
2022-09-30    10
2022-08-12    10
2022-06-03    10
2022-01-14     9
2022-08-05     9
2022-07-15     9
2022-02-11     9
2022-06-17     9
2022-04-08     8
2022-09-23     8
2022-04-01     8
2022-04-29     8
2022-07-29     8
2022-10-14     7
2022-10-28     7
2022-09-02     7
2022-05-06     6
2022-01-28     6
2022-09-09     5
2022-11-04     5
NaT            5
2022-04-15     5
2022-06-24     5
2022-05-20     5
2022-10-21     5
2022-02-25     4
2022-11-18     4
2022-02-17     4
2022-05-27     4
2022-10-04     4
2022-03-04     4
2022-07-01     4
2022-02-04     4
2022-07-08     4
2022-04-22     4
2022-02-18     4
2022-03-11     4
2022-03-25     4
2022-07-22     4
2022-10-06     3
2022-09-01     3
2022-01-21     3
2022-05-13     3
2022-08-19     3
2022-03-31     3
2022-01-22     3
2022-02-08     3
2022-02-03     3
2022-06-14     3
2022-06-23     3
2022-01-04     3
2022-01-15     3
2022-02-10     3
2022-02-02     3
2022-08-16    

In [21]:
import datetime as dt

movies_df_v2['release_weekday'] = movies_df_v2['release_date'].dt.isocalendar().day # weekday 5 is a friday


In [22]:
movies_df_v2 = movies_df_v2.drop(['release_month'], axis=1) #dropping the original month column and 
#creating one from datetime. The original release_month column has more nonsensical values


In [23]:
movies_df_v2['release_month'] = pd.DatetimeIndex(movies_df_v2['release_date']).month


In [24]:
movies_df_v2['release_month'].value_counts(dropna=False)

9.0     57
8.0     54
10.0    54
2.0     49
3.0     45
6.0     41
7.0     40
4.0     40
1.0     39
5.0     32
11.0    14
12.0     7
NaN      5
Name: release_month, dtype: int64

In [25]:
movies_df_v2['release_year'].value_counts(dropna=False)

2022    477
Name: release_year, dtype: int64

In [26]:
movies_df_v2['country_of_origin'].value_counts(dropna=False)

United States           426
United Kingdom           19
Canada                    4
Ireland                   4
Australia                 3
Japan                     2
China                     2
Italy                     2
Spain                     2
Germany                   2
Mexico                    2
United Arab Emirates      1
Puerto Rico               1
Portugal                  1
France                    1
South Korea               1
Poland                    1
Morocco                   1
Hungary                   1
Switzerland               1
Name: country_of_origin, dtype: int64

In [27]:
movies_df_v2['metascore'].value_counts(dropna=False)

[]    223
54     10
66     10
60     10
50      8
52      8
51      8
62      7
45      7
49      7
75      7
42      6
48      6
65      6
61      6
56      6
57      6
47      6
64      6
58      5
41      5
73      5
30      5
78      5
55      5
68      5
70      4
69      4
40      4
39      4
44      4
72      4
74      4
38      4
67      4
79      4
77      4
43      3
46      3
34      3
59      3
63      3
71      3
80      2
81      2
53      2
32      2
35      2
27      2
83      1
76      1
18      1
85      1
25      1
33      1
84      1
91      1
95      1
88      1
82      1
36      1
23      1
31      1
93      1
Name: metascore, dtype: int64

In [28]:
movies_df_v2['metascore']= movies_df_v2['metascore'].replace({
    '[]':'Not scored'    
})


In [29]:
def remove_currency(row): 
    if type(row) != float:
        row = row.replace('$', '')
        if '€' in row:
            row = row.replace('€', '').strip() # no cunversion needed as of date
        if 'CA' in row:
            row = row.replace('CA', '').strip()
            row = str(int(row)*0.73) 
        if '₹' in row:
            row = row.replace('₹', '').strip()
            row = str(int(row)*0.012) 
        if '£' in row:
            row = row.replace('£', '').strip()
            row = str(int(row)*1.14)
        if 'CN¥' in row:
            row = row.replace('CN¥', '').strip()
            row = str(int(row)*0.0067)
        if 'RUR' in row:
            row = row.replace('RUR', '').strip()
            row = str(int(row)*0.016)
             
    return row

movies_df_v2['budget_in_usd'] = movies_df_v2['budget_in_usd'].apply(remove_currency)

In [30]:
def remove_nonsensical(row): 
    if type(row) != float:
        if ':' in row:
            row = np.nan
            
    return row

In [31]:
movies_df_v2['opening_weekend_us_can_in_usd'] = movies_df_v2['opening_weekend_us_can_in_usd'].apply(remove_nonsensical)

In [32]:
movies_df_v2['gross_us_can_in_usd'] = movies_df_v2['gross_us_can_in_usd'].apply(remove_nonsensical)

In [33]:
movies_df_v2['gross_worldwide_in_usd'] = movies_df_v2['gross_worldwide_in_usd'].apply(remove_nonsensical)


In [34]:
movies_df_v2.isna().sum()

movie_id                           0
movie_imdb_link                    0
certificate                        0
runtime_in_mins                    0
genre                              0
imdb_rating                        0
number_of_votes                    0
metascore                          0
top_director                       0
release_year                       0
awards_link                        0
awards_total                       0
top_writer                         0
top_star_1                         0
top_star_2                         0
top_star_3                         0
num_user_reviews                   0
num_critic_reviews                20
release_date                       5
country_of_origin                  0
top_production_company             0
budget_in_usd                      0
opening_weekend_us_can_in_usd    136
gross_us_can_in_usd                4
gross_worldwide_in_usd            97
total_award_nominations            0
total_award_wins                   0
p

In [35]:
movies_df_v2.replace('',np.nan, inplace=True)
movies_df_v2.replace('[]',np.nan, inplace=True)

In [36]:
movies_df_v2.isna().sum()

movie_id                           0
movie_imdb_link                    0
certificate                      124
runtime_in_mins                    2
genre                              0
imdb_rating                        0
number_of_votes                    0
metascore                          0
top_director                       0
release_year                       0
awards_link                        0
awards_total                       0
top_writer                         1
top_star_1                         3
top_star_2                         3
top_star_3                         3
num_user_reviews                   0
num_critic_reviews                20
release_date                       5
country_of_origin                  0
top_production_company           270
budget_in_usd                    258
opening_weekend_us_can_in_usd    370
gross_us_can_in_usd              359
gross_worldwide_in_usd           278
total_award_nominations            0
total_award_wins                   0
p

In [37]:
cols_numeric = ['runtime_in_mins','num_user_reviews', 'metascore','budget_in_usd', 'opening_weekend_us_can_in_usd',
            'gross_us_can_in_usd', 'gross_worldwide_in_usd','total_award_nominations',
            'total_award_wins']

movies_df_v2[cols_numeric] = movies_df_v2[cols_numeric].apply(pd.to_numeric, errors='coerce', axis=1)


In [38]:
cols_object = ['release_year','release_month', 'release_weekday']

movies_df_v2[cols_object] = movies_df_v2[cols_object].astype(object)

In [39]:
movies_df_v2.isna().sum()

movie_id                           0
movie_imdb_link                    0
certificate                      124
runtime_in_mins                    2
genre                              0
imdb_rating                        0
number_of_votes                    0
metascore                        223
top_director                       0
release_year                       0
awards_link                        0
awards_total                       0
top_writer                         1
top_star_1                         3
top_star_2                         3
top_star_3                         3
num_user_reviews                   0
num_critic_reviews                20
release_date                       5
country_of_origin                  0
top_production_company           270
budget_in_usd                    258
opening_weekend_us_can_in_usd    370
gross_us_can_in_usd              359
gross_worldwide_in_usd           425
total_award_nominations            0
total_award_wins                   0
p

In [40]:
# Feature engineering

In [41]:
## Oscars proved to be useless for the model.

In [42]:
## Director

In [43]:
top_1000_movies_copy = top_1000_movies.copy()
best_directors =top_1000_movies_copy.pivot_table(index= ['Director'],  aggfunc = ['count'])
best_directors.columns = best_directors.columns.droplevel(0)
best_directors = best_directors.reset_index().rename_axis(None, axis=1)


In [44]:
best_directors = best_directors.drop(['Genre', 'Gross', 'IMDB_Rating',
       'Meta_score', 'No_of_Votes', 'Overview', 'Poster_Link', 'Released_Year',
       'Runtime', 'Series_Title', 'Star1', 'Star2', 'Star3', 'Star4'], axis=1)

best_directors = best_directors.rename(columns={'Director':'top_director',
                                    'Certificate':'is_among_best_director'})

best_directors['is_among_best_director'] = 'Y'

In [45]:
best_directors.shape


(548, 2)

In [46]:
movies_df_v2.shape

(477, 31)

In [47]:
movies_df_v10 = pd.merge(movies_df_v2, best_directors, how='left', on='top_director')

In [48]:
movies_df_v10.tail(10)

,movie_id,movie_imdb_link,certificate,runtime_in_mins,genre,imdb_rating,number_of_votes,metascore,top_director,release_year,awards_link,awards_total,top_writer,top_star_1,top_star_2,top_star_3,num_user_reviews,num_critic_reviews,release_date,country_of_origin,top_production_company,budget_in_usd,opening_weekend_us_can_in_usd,gross_us_can_in_usd,gross_worldwide_in_usd,total_award_nominations,total_award_wins,primary_genre,secondary_genre,release_weekday,release_month,is_among_best_director
467,tt13429928,https://www.imdb.com/title/tt13429928/?ref_=ad...,Not Rated,87.0,"Action, Adventure, History",3.2,210.0,NaN,Steven Luke,2022,https://www.imdb.com/title/tt13429928/awards/?...,0,Steven Luke,Hiram A. Murray,Andrew Stecker,Apostolos Gliarmis,11.0,5.0,2022-10-06,United States,NaN,NaN,NaN,NaN,NaN,0.0,0.0,Action,Adventure,4,10.0,NaN
468,tt11939970,https://www.imdb.com/title/tt11939970/?ref_=ad...,NaN,80.0,"Drama, Horror, Mystery",2.4,209.0,NaN,Kameron Hale,2022,https://www.imdb.com/title/tt11939970/awards/?...,Showing all 1 win and 3 nominations,Scott Hale,Miranda Nieman,Hayley Sunshine,Scott Hale,22.0,7.0,2022-01-18,United States,NaN,NaN,NaN,NaN,NaN,3.0,1.0,Drama,Horror,2,1.0,NaN
469,tt12907932,https://www.imdb.com/title/tt12907932/?ref_=ad...,NaN,82.0,"Horror, Thriller",2.5,208.0,NaN,Kipp Tribble,2022,https://www.imdb.com/title/tt12907932/awards/?...,0,Kipp Tribble,Andi Sweeney Blanco,Richard Siegelman,Kipp Tribble,10.0,11.0,2022-01-21,United States,NaN,NaN,NaN,NaN,NaN,0.0,0.0,Horror,Thriller,5,1.0,NaN
470,tt9093076,https://www.imdb.com/title/tt9093076/?ref_=adv...,NaN,72.0,Horror,3.7,207.0,NaN,Kurtis Spieler,2022,https://www.imdb.com/title/tt9093076/awards/?r...,0,Kurtis Spieler,Laura Dooling,Adrienne King,Frank Wihbey,13.0,15.0,2022-08-09,United States,NaN,NaN,NaN,NaN,NaN,0.0,0.0,Horror,Other,2,8.0,NaN
471,tt14555908,https://www.imdb.com/title/tt14555908/?ref_=ad...,NaN,86.0,Comedy,5.3,205.0,NaN,Andrew Nackman,2022,https://www.imdb.com/title/tt14555908/awards/?...,Showing all 0 wins and 1 nomination,Jake Greene,Ethan Dizon,Madison Wolfe,Bernard White,9.0,10.0,2022-05-24,United States,NaN,NaN,NaN,NaN,NaN,1.0,0.0,Comedy,Other,2,5.0,NaN
472,tt19511880,https://www.imdb.com/title/tt19511880/?ref_=ad...,NaN,NaN,Reality-TV,9.1,204.0,NaN,Diane Paloma Eskenazi,2022,https://www.imdb.com/title/tt19511880/awards/?...,0,Nino Dalakishvili,NaN,NaN,NaN,0.0,NaN,2022-05-01,United States,NaN,50000.0,NaN,NaN,NaN,0.0,0.0,Reality,TV,7,5.0,NaN
473,tt10696116,https://www.imdb.com/title/tt10696116/?ref_=ad...,NaN,92.0,Horror,7.9,204.0,NaN,John Ainslie,2022,https://www.imdb.com/title/tt10696116/awards/?...,0,John Ainslie,Kimberly Laferriere,Rogan Christopher,Janet Porter,3.0,26.0,2022-08-19,United States,NaN,NaN,NaN,NaN,NaN,0.0,0.0,Horror,Other,5,8.0,NaN
474,tt3447590,https://www.imdb.com/title/tt3447590/?ref_=adv...,Parental Guidance,117.0,"Comedy, Drama, Family",6.1,201.0,67.0,Matthew Warchus,2022,https://www.imdb.com/title/tt3447590/awards/?r...,0,Roald Dahl,Stephen Graham,Emma Thompson,Andrea Riseborough,15.0,15.0,2022-12-25,United Kingdom,Working Title Films,NaN,NaN,NaN,NaN,0.0,0.0,Comedy,Drama,7,12.0,Y
475,tt20861742,https://www.imdb.com/title/tt20861742/?ref_=ad...,NaN,78.0,Horror,4.5,200.0,NaN,Brendan Rudnicki,2022,https://www.imdb.com/title/tt20861742/awards/?...,Showing all 1 win and 4 nominations,Brendan Rudnicki,Walter Braithwaite,Dylan DeVane,Brent Downs,101.0,4.0,2022-07-29,United States,NaN,NaN,NaN,NaN,NaN,4.0,1.0,Horror,Other,5,7.0,NaN
476,tt12885770,https://www.imdb.com/title/tt12885770/?ref_=ad...,Adult,90.0,"Horror, Sci-Fi, Thriller",2.2,200.0,NaN,Lance Kawas,2022,https://www.imdb.com/title/tt12885770/awards/?...,0,Lance Kawas,Brande Roderick,Donald Cerrone,Kelly Lynn Reiter,9.0,3.0,2022-10-11,United States,NaN,NaN,NaN,NaN,NaN,0.0,0.0,Horror,Sci,2,10.0,NaN


In [49]:
movies_df_v10_bud = movies_df_v10[~(
    movies_df_v10['budget_in_usd'] < 10000)]

movies_df_v10_gross_uscan = movies_df_v10_bud[~(
    movies_df_v10['gross_us_can_in_usd'] < 2000)]

/var/folders/33/p_3l01b14g96rn22vzwly2g00000gn/T/ipykernel_64253/2676440861.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  movies_df_v10_gross_uscan = movies_df_v10_bud[~(


In [50]:
movies_df_v10_gross_uscan.isna().sum()

movie_id                           0
movie_imdb_link                    0
certificate                      122
runtime_in_mins                    2
genre                              0
imdb_rating                        0
number_of_votes                    0
metascore                        219
top_director                       0
release_year                       0
awards_link                        0
awards_total                       0
top_writer                         1
top_star_1                         3
top_star_2                         3
top_star_3                         3
num_user_reviews                   0
num_critic_reviews                18
release_date                       5
country_of_origin                  0
top_production_company           263
budget_in_usd                    258
opening_weekend_us_can_in_usd    363
gross_us_can_in_usd              352
gross_worldwide_in_usd           418
total_award_nominations            0
total_award_wins                   0
p

In [51]:
# I will continue with a filtered data where the financial columns are not null.

mask1 = movies_df_v10_gross_uscan['gross_worldwide_in_usd'].notnull()
mask2 = movies_df_v10_gross_uscan['opening_weekend_us_can_in_usd'].notnull()
mask3 = movies_df_v10_gross_uscan['budget_in_usd'].notnull()

movies_filtered = movies_df_v10_gross_uscan[mask1 & mask2 & mask3].copy()

In [52]:
movies_filtered.shape

(52, 32)

In [53]:
movies_filtered.isna().sum()

movie_id                          0
movie_imdb_link                   0
certificate                       0
runtime_in_mins                   0
genre                             0
imdb_rating                       0
number_of_votes                   0
metascore                         0
top_director                      0
release_year                      0
awards_link                       0
awards_total                      0
top_writer                        0
top_star_1                        0
top_star_2                        0
top_star_3                        0
num_user_reviews                  0
num_critic_reviews                0
release_date                      1
country_of_origin                 0
top_production_company            5
budget_in_usd                     0
opening_weekend_us_can_in_usd     0
gross_us_can_in_usd               0
gross_worldwide_in_usd            0
total_award_nominations           0
total_award_wins                  0
primary_genre               

In [54]:
movies_filtered['release_weekday'].value_counts()

5    49
4     2
Name: release_weekday, dtype: int64

In [55]:
movies_filtered['release_weekday'] = movies_filtered['release_weekday'].fillna(5).astype(object)


In [56]:
movies_filtered['release_month'].value_counts()

4.0     10
2.0      7
9.0      7
3.0      4
7.0      4
8.0      4
6.0      4
10.0     4
5.0      3
1.0      2
11.0     2
Name: release_month, dtype: int64

In [57]:
movies_filtered['release_month'] = movies_filtered['release_month'].fillna(4.0).astype(object)
movies_filtered['release_month'].value_counts()


4.0     11
2.0      7
9.0      7
3.0      4
7.0      4
8.0      4
6.0      4
10.0     4
5.0      3
1.0      2
11.0     2
Name: release_month, dtype: int64

In [58]:
def group_prod_company(df):

    major_prod_company = ['Universal Pictures', 'Columbia Pictures', 'Warner Bros.', 'Paramount Pictures']
    df['top_production_company_grouped'] = '' #creating an empty column
    
    for i in df.index:
        if df['top_production_company'][i] not in major_prod_company:
            df['top_production_company_grouped'][i] = 'Other'      
         
        else:
            df['top_production_company_grouped'][i] = df['top_production_company'][i]
  
    return df



In [59]:
movies_filtered_v1 = group_prod_company(movies_filtered)

/var/folders/33/p_3l01b14g96rn22vzwly2g00000gn/T/ipykernel_64253/2722901165.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['top_production_company_grouped'][i] = df['top_production_company'][i]
/var/folders/33/p_3l01b14g96rn22vzwly2g00000gn/T/ipykernel_64253/2722901165.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['top_production_company_grouped'][i] = 'Other'


In [60]:
movies_filtered_v1['top_production_company_grouped'].value_counts(dropna=False)

Other                 37
Universal Pictures     6
Paramount Pictures     4
Warner Bros.           3
Columbia Pictures      2
Name: top_production_company_grouped, dtype: int64

In [61]:
movies_filtered_v1['top_production_company']= movies_filtered_v1['top_production_company'].fillna('Other')

In [62]:
movies_filtered_v1['top_production_company'].isna().sum()

0

In [63]:
def group_country(df):

    df['country_of_origin_grouped'] = '' #creating an empty column
    
    for i in df.index:
        if df['country_of_origin'][i] != 'United States':
            df['country_of_origin_grouped'][i] = 'Other'      
         
        else:
            df['country_of_origin_grouped'][i] = df['country_of_origin'][i]
  
    return df



In [64]:
movies_filtered_v2 = group_country(movies_filtered_v1)

/var/folders/33/p_3l01b14g96rn22vzwly2g00000gn/T/ipykernel_64253/2431420963.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['country_of_origin_grouped'][i] = df['country_of_origin'][i]
/var/folders/33/p_3l01b14g96rn22vzwly2g00000gn/T/ipykernel_64253/2431420963.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['country_of_origin_grouped'][i] = 'Other'


In [65]:
movies_filtered_v2['primary_genre']= movies_filtered_v2['primary_genre'].replace({'Fantasy': 'Other',
        'Thriller': 'Other','Romance': 'Other', 'Music': 'Other', 'Mystery': 'Other', 'Musical': 'Other',
        'Sci':'Other' })

movies_filtered_v2['primary_genre'].value_counts(dropna=False)


Action       22
Animation     6
Comedy        6
Horror        5
Drama         4
Crime         3
Biography     2
Adventure     2
Other         2
Name: primary_genre, dtype: int64

In [66]:
movies_filtered_v2['secondary_genre']= movies_filtered_v2['secondary_genre'].replace({'Mystery': 'Other',
        'Fantasy': 'Other','Family': 'Other', 'Biography': 'Other', 'Action': 'Other', 'Sci': 'Other',
        'History': 'Other','Music': 'Other', 'Sport': 'Other', 'War': 'Other', 'Musical': 'Other', 
        'Western': 'Other', 'Fi': 'Other'})

movies_filtered_v2['secondary_genre'].value_counts(dropna=False)


Adventure    15
Drama        11
Other        11
Thriller      6
Crime         3
Comedy        3
Romance       2
Horror        1
Name: secondary_genre, dtype: int64

In [67]:
movies_filtered_v2['certificate']=movies_filtered_v2['certificate'].fillna('Not Rated')
movies_filtered_v2['certificate'].value_counts(dropna=False)

+13/14               24
Adult                20
Parental Guidance     8
Name: certificate, dtype: int64

In [68]:
movies_filtered_v2['num_critic_reviews']=movies_filtered_v2['num_critic_reviews'].fillna(0)

In [69]:
movies_filtered_v2.isna().sum().sum()

42

In [70]:
movies_filtered_v2['num_critic_reviews']=movies_filtered_v2['num_critic_reviews'].fillna(0)

In [71]:
movies_filtered_v2.columns

Index(['movie_id', 'movie_imdb_link', 'certificate', 'runtime_in_mins',
       'genre', 'imdb_rating', 'number_of_votes', 'metascore', 'top_director',
       'release_year', 'awards_link', 'awards_total', 'top_writer',
       'top_star_1', 'top_star_2', 'top_star_3', 'num_user_reviews',
       'num_critic_reviews', 'release_date', 'country_of_origin',
       'top_production_company', 'budget_in_usd',
       'opening_weekend_us_can_in_usd', 'gross_us_can_in_usd',
       'gross_worldwide_in_usd', 'total_award_nominations', 'total_award_wins',
       'primary_genre', 'secondary_genre', 'release_weekday', 'release_month',
       'is_among_best_director', 'top_production_company_grouped',
       'country_of_origin_grouped'],
      dtype='object')

In [72]:
movies_filtered_v2['is_among_best_director']= movies_filtered_v2['is_among_best_director'].fillna('N')

In [73]:
movies_filtered_v2.isna().sum().sum()

1

In [74]:
movies_filtered_v2.to_csv('movies_2022_treated.csv')
